In [6]:
from pathlib import Path
import pandas as pd

# pip install git+https://github.com/boudinfl/pke.git
# pke relies on spacy (>= 3.2.3) for text processing and requires models to be installed:
# # download the english model
# python -m spacy download en_core_web_sm
import pke

import string

In [7]:
all_df = pd.read_csv("dest/all_dataset.csv", sep=";")
all_df = all_df[all_df["rubrics"].str.contains('Образование_онлайн_курсы')]

In [8]:
text = " ".join(list(all_df['text'].dropna().astype("str")))[:1000000]

In [ ]:
# 1. create a MultipartiteRank extractor.
extractor = pke.unsupervised.MultipartiteRank()

stoplist = list(string.punctuation)
stoplist += pke.lang.stopwords.get('ru')

# 2. load the content of the document.
extractor.load_document(input=text,
                        language="ru",
                        stoplist=stoplist)

# 3. select the longest sequences of nouns and adjectives, that do
#    not contain punctuation marks or stopwords as candidates.
pos = {'NOUN', 'PROPN', 'ADJ'}
extractor.candidate_selection(pos=pos)

# 4. build the Multipartite graph and rank candidates using random
#    walk, alpha controls the weight adjustment mechanism, see
#    TopicRank for threshold/method parameters.
extractor.candidate_weighting(alpha=1.1,
                              threshold=0.74,
                              method='average')

# 5. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=10)


In [ ]:
keyphrases